In [1]:
import pandas as pd
import numpy as np 
import scipy.optimize as optimize
import random

Consider the square cyclic graph 1--2--3--4--1. A precision matrix that satisfies this structure is given by

In [2]:
d = 4
Theta = np.matrix([
[10 , 0.5  , 0 , 0.5],
[0.5 , 10 , 0.5 , 0 ],
[0 , 0.5 , 10 , 0.5],
[0.5 , 0 , 0.5 , 10]
])

print("This is the true precision matrix  \n", Theta)

This is the true precision matrix  
 [[10.   0.5  0.   0.5]
 [ 0.5 10.   0.5  0. ]
 [ 0.   0.5 10.   0.5]
 [ 0.5  0.   0.5 10. ]]


In [3]:
Sigma = np.linalg.inv(Theta) 
print("This is the true covariance matrix  \n", Sigma)

This is the true covariance matrix  
 [[ 0.10050505 -0.00505051  0.00050505 -0.00505051]
 [-0.00505051  0.10050505 -0.00505051  0.00050505]
 [ 0.00050505 -0.00505051  0.10050505 -0.00505051]
 [-0.00505051  0.00050505 -0.00505051  0.10050505]]


We first generate a sample of size $N=10000$ from a centered multivariate Guassian distribution with covariance matrix $\Sigma$

In [4]:
N = 10000
d = 4
mean = np.zeros(4)
random.seed(7)
X = np.random.multivariate_normal(mean ,Sigma , N)
X = np.transpose(X)
print(X)
####To calculate the covariance matrix using np.cov, we need to reshape X into a matrix where each row represnets a variable and each column a single observation
X.shape

[[-0.79051174  0.18334081  0.30050765 ... -0.19815123  0.11462431
   0.4796144 ]
 [ 0.06549071  0.30895145 -0.20687958 ... -0.13019194  0.5135878
   0.51564668]
 [ 0.67249927 -0.26054063  0.09772485 ...  0.01580398  0.02297572
   0.12713282]
 [-0.72422374  0.14396419 -0.1340321  ...  0.13793598  0.36966666
   0.29853747]]


(4, 10000)

In [5]:
EstimSigma = np.cov(X)
EstimTheta =  np.linalg.inv(EstimSigma)
print("This is the sample covariance matrix is  \n", EstimSigma)
print("This is the estimated precision matrix is  \n", EstimTheta)

This is the sample covariance matrix is  
 [[ 0.10009737 -0.00487598  0.00017038 -0.00438293]
 [-0.00487598  0.09949922 -0.00570153 -0.00051381]
 [ 0.00017038 -0.00570153  0.09934195 -0.00539141]
 [-0.00438293 -0.00051381 -0.00539141  0.1014597 ]]
This is the estimated precision matrix is  
 [[10.03354239  0.4959632   0.03501681  0.43780882]
 [ 0.4959632  10.10868924  0.58494463  0.10370029]
 [ 0.03501681  0.58494463 10.12920659  0.54272473]
 [ 0.43780882  0.10370029  0.54272473  9.9044071 ]]


We will now compute the maximum likelihood estimator by maximizing the equivalent version of the loglikelihood function defined on Chapter 2

In [6]:
import scipy.optimize as optimize
def construct_symmetric_matrix(Theta):
    Theta_matrix = np.zeros((4 , 4))
    upper_idx = np.triu_indices(4)
    Theta_matrix[upper_idx] = Theta
    Theta_matrix = Theta_matrix + Theta_matrix.T - np.diag(np.diag(Theta_matrix)) 
    return(Theta_matrix)

def log_likelihood(Theta , EstimSigma) :
    Theta_matrix  = construct_symmetric_matrix(Theta)
    det_Theta = np.linalg.det(Theta_matrix) 
    if det_Theta <= 0 :
        return -np.inf
    product_STheta = np.matmul(EstimSigma, Theta_matrix)
    trace_STheta =  product_STheta.trace()
    return(np.log(det_Theta) - trace_STheta)

initial_guess = [4 , 1 , 0.5 , 0.2 , 3 , 0.3 , 0.1 , 2 , 0.4 , 1.5]
random.seed(7)
result = optimize.minimize(lambda x: -log_likelihood(x , EstimSigma) , np.array(initial_guess) ).x
mle =  construct_symmetric_matrix(result)
print("The maximum likelihood estimator of the precision matrix is given by \n", mle)

The maximum likelihood estimator of the precision matrix is given by 
 [[10.03354921  0.49591092  0.03484018  0.43801696]
 [ 0.49591092 10.10870391  0.58509876  0.10358512]
 [ 0.03484018  0.58509876 10.12922799  0.54271461]
 [ 0.43801696  0.10358512  0.54271461  9.90440235]]


Note that even with a larger sample size, the estimates $\hat{\Theta}_{13}$ and $\hat{\Theta}_{24}$ will never be equal exactly to $0$. This is illustrated on the following code with smaple size going from $10^2$ to $10^6$.

In [9]:
power = np.arange(2 , 7 , 1)
size = []
for p in power:
    size.append(pow(10 , p))
size = np.array(size)    
print(size)

def main(N):
    random.seed(7)
    X = np.random.multivariate_normal(mean ,Sigma , N)
    X = np.transpose(X)
    EstimSigmaN = np.cov(X)
    result = optimize.minimize(lambda x: -log_likelihood(x , EstimSigmaN) , np.array(initial_guess) ).x
    mle =  construct_symmetric_matrix(result)
    print("With sample size N = ", N, ", the maximum likelihood estimator of the precision matrix is given by \n", mle)

for N in size:
    main(N)

[    100    1000   10000  100000 1000000]
With sample size N =  100 , the maximum likelihood estimator of the precision matrix is given by 
 [[11.45020123  0.90636239 -0.34762544 -1.41623308]
 [ 0.90636239  8.55222269 -1.10410326 -0.96673037]
 [-0.34762544 -1.10410326 11.34535961  2.30935492]
 [-1.41623308 -0.96673037  2.30935492 10.63835465]]
With sample size N =  1000 , the maximum likelihood estimator of the precision matrix is given by 
 [[10.06569805  1.04907303  0.51052225  0.56729925]
 [ 1.04907303 10.5589906   0.91844138  0.31454863]
 [ 0.51052225  0.91844138  9.33619215  0.46441708]
 [ 0.56729925  0.31454863  0.46441708  9.69233254]]
With sample size N =  10000 , the maximum likelihood estimator of the precision matrix is given by 
 [[ 9.78236191  0.55471038  0.01233441  0.61516589]
 [ 0.55471038 10.10584601  0.26308159 -0.05726975]
 [ 0.01233441  0.26308159  9.93334117  0.62587329]
 [ 0.61516589 -0.05726975  0.62587329  9.94711745]]
With sample size N =  100000 , the maximum 